In [42]:
import socket
import time
from pymavlink import mavutil

In [6]:
class MessengerClient:
    def __init__(self , host="localhost" , port=42425):
        self.host = host
        self.port = port
        self.s = socket.socket()
    
    def connect(self):
        self.s.connect((self.host,self.port))
    
    def relay_msg(self , function):
        try:
            while True:
                data = self.s.recv(1024)
                if data is None:
                    break
                function(data)
        except KeyboardInterrupt:
            print("Breaking")

In [7]:
port = 42425
client = MessengerClient(port = port)
client.connect()

In [8]:
#client.s.close()

In [9]:
from dronekit_sitl import SITL , start_default
sitl = start_default()

Starting copter simulator (SITL)
SITL already Downloaded and Extracted.
Ready to boot.


In [10]:
connection_string = sitl.connection_string()
print("Connecting to vehicle on: %s" % (connection_string,))

Connecting to vehicle on: tcp:127.0.0.1:5760


In [11]:
#mavproxy.exe --master tcp:127.0.0.1:5760 --sitl 127.0.0.1:5501 --out 127.0.0.1:14550 --out 127.0.0.1:14551              

In [12]:
from dronekit import connect, VehicleMode

In [13]:
vehicle = connect("127.0.0.1:14550", wait_ready=True)

CRITICAL:autopilot:APM:Copter V3.3 (d6053245)
CRITICAL:autopilot:Frame: QUAD


In [14]:
print(vehicle.battery)

Battery:voltage=12.587,current=0.0,level=67


In [15]:
print(vehicle.is_armable)

True


In [16]:
print(vehicle.mode)

VehicleMode:LAND


In [48]:
class DroneControl:
    
    def __init__(self , vehicle, messenger_client , time_interval=1):
        self.vehicle = vehicle
        self.messenger_client = messenger_client
        self.time_interval = time_interval
    
    def send_ned_velocity(self,velocity_x, velocity_y, velocity_z, duration):
        msg = vehicle.message_factory.set_position_target_local_ned_encode(
            0,       # time_boot_ms (not used)
            0, 0,    # target system, target component
            mavutil.mavlink.MAV_FRAME_LOCAL_NED, # frame
            0b0000111111000111, # type_mask (only speeds enabled)
            0, 0, 0, # x, y, z positions (not used)
            velocity_x, velocity_y, velocity_z, # x, y, z velocity in m/s
            0, 0, 0, # x, y, z acceleration (not supported yet, ignored in GCS_Mavlink)
            0, 0)    # yaw, yaw_rate (not supported yet, ignored in GCS_Mavlink) 

        # send command to vehicle on 1 Hz cycle
        for x in range(0,duration):
            vehicle.send_mavlink(msg)
            time.sleep(1) 
    
    def steer(self,data):
        if str(data).find("LEFT") > -1:
            self.send_ned_velocity(0,0.5,0,self.time_interval)
        if str(data).find("RIGHT") > -1:
            self.send_ned_velocity(0,-0.5,0,self.time_interval)
        if str(data).find("ASCEND") > -1:
            self.send_ned_velocity(0.5,0,0,self.time_interval)
        if str(data).find("DESCEND") > -1:
            self.send_ned_velocity(-0.5,0,0,self.time_interval)
        print(data)
    
    def set_to_guided(self):
        self.vehicle.mode = VehicleMode("GUIDED")
        self.vehicle.armed= True
        self.vehicle.simple_takeoff(25)
        
    def gesture_control(self):
        self.messenger_client.relay_msg(self.steer)
    

In [49]:
drone_control = DroneControl(vehicle , client , 1)

In [72]:
drone_control.set_to_guided()

ERROR:autopilot:ARMING MOTORS
CRITICAL:autopilot:Initialising APM...


In [ ]:
drone_control.gesture_control()

b'NONENONENONENONENONENONENONENONENONENONENONENONENONENONENONENONENONENONENONENONE'
b'NONE'
b'NONE'
b'NONE'
b'NONE'
b'NONE'
b'NONE'
b'NONE'
b'NONE'
b'NONE'
b'NONE'
b'NONE'
b'NONE'
b'NONE'
b'NONE'
b'NONE'
b'NONE'
b'NONE'
b'NONE'
b'LEFT'
b'LEFT'
b'LEFT'
b'LEFT'
b'LEFT'
b'LEFT'
b'LEFT'
b'LEFT'
b'LEFT'
b'LEFT'
b'LEFT'
b'LEFT'
b'LEFT'
b'LEFT'
b'LEFT'
b'LEFT'
b'LEFT'
b'LEFT'
b'LEFT'
b'LEFT'
b'NONE'
b'LEFT'
b'LEFT'
b'LEFT'
b'ASCEND'
b'ASCEND'
b'ASCEND'
b'RIGHT'
b'ASCEND'
b'ASCEND'
b'ASCEND'
b'ASCEND'
b'ASCEND'
b'ASCEND'
b'ASCEND'
b'ASCEND'
b'ASCEND'
b'ASCEND'
b'ASCEND'
b'ASCEND'
b'ASCEND'
b'ASCEND'
b'ASCEND'
b'ASCEND'
b'ASCEND'
b'ASCEND'
b'ASCEND'
b'ASCEND'
b'ASCEND'
b'RIGHT'
b'RIGHT'
b'RIGHT'
b'RIGHT'
b'RIGHT'
b'RIGHT'
b'RIGHT'
b'RIGHT'
b'RIGHT'
b'RIGHT'
b'RIGHT'
b'RIGHT'
b'RIGHT'
b'RIGHT'
b'RIGHT'
b'RIGHT'
b'RIGHT'
b'RIGHT'
b'RIGHT'
b'DESCEND'
b'DESCEND'
b'DESCEND'
b'RIGHT'
b'DESCEND'
b'DESCEND'
b'DESCEND'
b'DESCEND'
b'DESCEND'
b'DESCEND'
b'DESCEND'
b'DESCEND'
b'DESCEND'
b'DESCEND'
b'DESCEN

ERROR:autopilot:DISARMING MOTORS


b'NONE'
b'NONE'
b'NONE'
b'NONE'
b'NONE'
b'NONE'
b'NONE'
b'NONE'
b'NONE'
b'NONE'
b'NONE'
b'NONE'
b'NONE'
b'NONE'
b'NONE'
b'NONE'
b'NONE'
b'NONE'
b'NONE'
b'NONE'
b'NONE'
b'NONE'
b'NONE'
b'NONE'
b'NONE'
b'NONE'
b'NONE'
b'NONE'
b'NONE'
b'NONE'
b'NONE'
b'NONE'
b'NONE'
b'NONE'
b'NONE'
b'NONE'
b'NONE'
b'NONE'
b'NONE'
b'NONE'
b'NONE'
b'NONE'
b'NONE'
b'NONE'
b'NONE'
b'NONE'
b'NONE'
b'NONE'
b'NONE'
b'NONE'
b'NONE'
b'NONE'
b'NONE'
b'NONE'
b'NONE'
b'NONE'
b'NONE'
b'NONE'
